# Partie III - Application du machine learning pour la fonction d'évaluation

## Deuxième modèle : Arbre de décision

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Charger les fichiers CSV pour l'entraînement
gagnants = pd.read_csv("chemin/vers/gagnants.csv", header=None)  # Remplace avec ton chemin
perdants = pd.read_csv("chemin/vers/perdants.csv", header=None)

# Ajouter la colonne cible (1 pour gagnant, -1 pour perdant)
gagnants["score"] = 1
perdants["score"] = -1

# Fusionner les données
data = pd.concat([gagnants, perdants], ignore_index=True)

# Séparer les features et labels
X = data.iloc[:, :-1].values  # Les configurations de plateau
y = data["score"].values  # Les scores

# Diviser en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner l'arbre de décision
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Évaluer le modèle
y_pred = clf.predict(X_test)
print("Précision du modèle :", accuracy_score(y_test, y_pred))

# Définition du plateau Fanorona Telo (3x3 intersections)
class FanoronaTelo:
    def __init__(self):
        self.board = [0] * 9  # 0 = vide, 1 = blanc, -1 = noir
        self.current_player = 1  # IA
    
    def reset(self):
        self.board = [0] * 9
        self.current_player = 1
    
    def get_valid_moves(self):
        return [i for i in range(9) if self.board[i] == 0]
    
    def make_move(self, pos):
        if self.board[pos] == 0:
            self.board[pos] = self.current_player
            self.current_player *= -1
            return True
        return False
    
    def is_winning(self, player):
        winning_positions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Lignes
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Colonnes
            [0, 4, 8], [2, 4, 6]             # Diagonales
        ]
        for pos in winning_positions:
            if all(self.board[i] == player for i in pos):
                return True
        return False
    
    def is_draw(self):
        return all(x != 0 for x in self.board)
    
    def get_state(self):
        return self.board.copy()

# Génération de données pour entraîner l'arbre de décision
def generate_training_data(num_games=1000):
    X, y = [], []
    game = FanoronaTelo()
    
    for _ in range(num_games):
        game.reset()
        for _ in range(9):
            valid_moves = game.get_valid_moves()
            if not valid_moves:
                break
            move = random.choice(valid_moves)
            state_before = game.get_state()
            game.make_move(move)
            state_after = game.get_state()
            
            if game.is_winning(1):
                score = 1
            elif game.is_winning(-1):
                score = -1
            else:
                score = 0
            
            X.append(state_before)
            y.append(score)
    
    return np.array(X), np.array(y)

# Entraînement du modèle
X_train, y_train = generate_training_data(5000)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Fonction pour choisir le meilleur coup grâce à l'arbre de décision
def choose_best_move(game):
    best_move = None
    best_score = -float('inf') if game.current_player == 1 else float('inf')
    
    for move in game.get_valid_moves():
        temp_board = game.get_state()
        temp_board[move] = game.current_player
        score = clf.predict([temp_board])[0]
        
        if (game.current_player == 1 and score > best_score) or (game.current_player == -1 and score < best_score):
            best_score = score
            best_move = move
    
    return best_move

# Simulation d'une partie
print("Début du jeu Fanorona Telo avec AI")
game = FanoronaTelo()
while True:
    print("Plateau actuel:", game.board)
    if game.current_player == 1:
        move = choose_best_move(game)
        print(f"L'IA Blanc joue en position {move}")
    else:
        move = random.choice(game.get_valid_moves())
        print(f"L'IA Noir joue en position {move}")
    
    game.make_move(move)
    
    if game.is_winning(1):
        print("Les Blancs gagnent!")
        break
    elif game.is_winning(-1):
        print("Les Noirs gagnent!")
        break
    elif game.is_draw():
        print("Match nul!")
        break
